# Set up

In [8]:
import duckdb
import pandas as pd
from IPython.display import display
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from eda_support_functions import *

# Suppress all warnings
import warnings

warnings.filterwarnings('ignore')

In [9]:
full__regression__none = pd.read_csv('../data/ml_ready_data/full__regression__with_outliers__none.csv')

In [10]:
full__regression__none

,movie_id,original_language,runtime,ageCert,quarter,month,year,is_released_US,is_released_CN,is_released_FR,is_released_GB,is_released_JP,budget_usd_adj,revenue_usd_adj,is_outlier
0,340666,en,117,R,4,11,2016,True,False,True,True,False,2.856498e+07,3.848256e+07,False
1,14347,en,100,R,1,2,1984,True,False,True,True,False,2.639382e+07,5.467794e+07,False
2,14351,en,95,PG13,3,9,2006,True,False,False,False,False,8.766228e+06,1.059454e+07,False
3,14353,en,98,U,4,11,2008,False,False,False,False,False,1.202941e+07,2.662405e+05,True
4,340674,en,93,U,4,10,2016,False,False,False,False,False,5.688104e+06,2.693487e+04,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7639,14342,en,90,PG13,2,5,1998,True,False,False,False,False,5.608012e+07,1.147185e+07,False
7640,208763,ru,128,U,1,1,2014,False,False,True,False,False,3.346450e+07,4.452338e+07,True
7641,54952,pt,90,U,1,2,2007,False,False,False,False,False,7.347812e+05,4.163182e+05,True
7642,14337,en,77,U,4,10,2004,False,False,True,True,True,1.129123e+04,8.798065e+05,True


In [11]:
full__regression__none['ratio'] = full__regression__none['budget_usd_adj'] / full__regression__none['revenue_usd_adj']

In [12]:
display(full__regression__none[~full__regression__none['is_outlier']].sort_values(by='ratio', ascending=False)[['movie_id', 'ratio', 'budget_usd_adj','revenue_usd_adj']].head(100).style.format({'budget_usd_adj': "${:,.0f}",'revenue_usd_adj': "${:,.0f}"}))

,movie_id,ratio,budget_usd_adj,revenue_usd_adj
2230,226749,31.735957,"$1,402,003","$44,177"
6943,75889,30.446432,"$36,059,408","$1,184,356"
5204,70695,27.877478,"$5,589,427","$200,500"
7563,54580,27.322031,"$6,774,919","$247,965"
5166,26850,27.113130,"$13,422,996","$495,074"
6685,51515,26.747841,"$80,537,974","$3,011,008"
6351,11468,26.599754,"$30,487,137","$1,146,144"
5094,9968,25.837886,"$21,842,437","$845,365"
5332,47871,25.547597,"$80,449,373","$3,149,000"
6593,29973,25.308089,"$52,428,482","$2,071,610"


In [13]:
display(full__regression__none[~full__regression__none['is_outlier']].sort_values(by='ratio', ascending=True)[['movie_id', 'ratio', 'budget_usd_adj','revenue_usd_adj']].head(40).style.format({'budget_usd_adj': "${:,.0f}",'revenue_usd_adj': "${:,.0f}"}))

,movie_id,ratio,budget_usd_adj,revenue_usd_adj
6044,11050,0.073785,"$24,474,056","$331,696,116"
7054,293660,0.074089,"$73,634,169","$993,854,785"
2128,18785,0.074577,"$49,709,700","$666,551,459"
893,546,0.074904,"$1,560,174","$20,828,900"
2465,19610,0.075000,"$32,130,264","$428,403,515"
6723,12102,0.075287,"$33,575,978","$445,972,927"
2543,527641,0.075594,"$8,342,873","$110,364,297"
3761,180147,0.075661,"$2,615,951","$34,574,562"
5400,27323,0.075902,"$917,777","$12,091,554"
372,15389,0.076001,"$524,544","$6,901,820"
